﻿# 多层感知器 MNIST
---
在此 notebook 中，我们将训练 MLP 对 [MNIST 手写数字数据库](http://yann.lecun.com/exdb/mnist/)中的图像进行分类。

流程将分为以下几步：
>1.加载和可视化数据
2.定义神经网络
3.训练模型
4.评估已训练模型在测试数据集上的性能！

在开始之前，我们需要导入处理数据和使用 PyTorch 所必需的库。

In [ ]:
# import libraries
import torch
import numpy as np

---
## 加载和可视化[数据](http://pytorch.org/docs/stable/torchvision/datasets.html)

下载过程可能需要几分钟，你应该会看到数据加载进度。如果你想一次加载更多数据，还可以选择更改 `batch_size`。

此单元格将为每个数据集创建数据加载器。

In [ ]:
from torchvision import datasets
import torchvision.transforms as transforms

# number of subprocesses to use for data loading
num_workers = 0
# how many samples per batch to load
batch_size = 20

# convert data to torch.FloatTensor
transform = transforms.ToTensor()

# choose the training and test datasets
train_data = datasets.MNIST(root='data', train=True,
                                   download=True, transform=transform)
test_data = datasets.MNIST(root='data', train=False,
                                  download=True, transform=transform)

# prepare data loaders
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
    num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, 
    num_workers=num_workers)

### 可视化一批训练数据

分类任务的第一步是查看数据，确保加载正确，然后初步观察数据中的规律。

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
    
# obtain one batch of training images
dataiter = iter(train_loader)
images, labels = dataiter.next()
images = images.numpy()

# plot the images in the batch, along with the corresponding labels
fig = plt.figure(figsize=(25, 4))
for idx in np.arange(20):
    ax = fig.add_subplot(2, 20/2, idx+1, xticks=[], yticks=[])
    ax.imshow(np.squeeze(images[idx]), cmap='gray')
    # print out the correct label for each image
    # .item() gets the value contained in a Tensor
    ax.set_title(str(labels[idx].item()))

### 更详细地查看图像

In [ ]:
img = np.squeeze(images[1])

fig = plt.figure(figsize = (12,12)) 
ax = fig.add_subplot(111)
ax.imshow(img, cmap='gray')
width, height = img.shape
thresh = img.max()/2.5
for x in range(width):
    for y in range(height):
        val = round(img[x][y],2) if img[x][y] !=0 else 0
        ax.annotate(str(val), xy=(y,x),
                    horizontalalignment='center',
                    verticalalignment='center',
                    color='white' if img[x][y]<thresh else 'black')

---
## 定义网络[结构](http://pytorch.org/docs/stable/nn.html)

该结构需要将每个图像的 784 维像素值张量当做输入，并生成长度为 10（表示类别数量）的张量，表示输入图像的类别分数。这个示例使用了两个隐藏层和丢弃层，以避免过拟合。

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

## TODO: Define the NN architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # linear layer (784 -> 1 hidden node)
        self.fc1 = nn.Linear(28 * 28, 1)

    def forward(self, x):
        # flatten image input
        x = x.view(-1, 28 * 28)
        # add hidden layer, with relu activation function
        x = F.relu(self.fc1(x))
        return x

# initialize the NN
model = Net()
print(model)

###  指定[损失函数](http://pytorch.org/docs/stable/nn.html#loss-functions)和[优化器](http://pytorch.org/docs/stable/optim.html)

对于分类任务来说，建议使用交叉熵损失。如果你查看上述链接文档，就可以发现 PyTorch 的交叉熵函数向输出层应用了 softmax 函数，然后计算对数损失。

In [ ]:
## TODO: Specify loss and optimization functions

# specify loss function
criterion = None

# specify optimizer
optimizer = None

---
## 训练网络

以下注释描述了如何用一批数据训练网络：
1.清空所有已优化变量的梯度
2.前向传播：通过向模型传入输入，计算预测输出。
3.计算损失
4.反向传播：计算损失相对于模型参数的梯度
5.执行一个优化步骤（参数更新）
6.更新平均训练损失

以下循环训练了 30 个周期，当然，你也可以选定自己想训练的周期数。暂时，我们建议训练 20-30 个周期。在训练过程中，请观察训练损失的值如何逐渐降低。我们希望它能降低，同时避免在训练数据上过拟合。

In [ ]:
# number of epochs to train the model
n_epochs = 30  # suggest training between 20-50 epochs

model.train() # prep model for training

for epoch in range(n_epochs):
    # monitor training loss
    train_loss = 0.0
    
    ###################
    # train the model #
    ###################
    for data, target in train_loader:
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the loss
        loss = criterion(output, target)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update running training loss
        train_loss += loss.item()*data.size(0)
        
    # print training statistics 
    # calculate average loss over an epoch
    train_loss = train_loss/len(train_loader.dataset)

    print('Epoch: {} \tTraining Loss: {:.6f}'.format(
        epoch+1, 
        train_loss
        ))

---
## 测试训练的网络

最后，使用之前未见过的**测试数据**测试最佳模型，并评估其性能。用未见过的数据测试模型可以很好地检查模型的泛化效果。此外，建议在此分析过程中循序渐进地操作，看看模型对于每个类别的效果如何，并查看总体损失和准确率。

#### `model.eval()`

`model.eval(` 会将模型中的所有层级设为评估模式。这样会影响到丢弃层等层级，但是应该会允许所有节点接受评估。丢弃层是指会在训练期间按照某个概率关闭节点的层级。

In [ ]:
# initialize lists to monitor test loss and accuracy
test_loss = 0.0
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))

model.eval() # prep model for *evaluation*

for data, target in test_loader:
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model(data)
    # calculate the loss
    loss = criterion(output, target)
    # update test loss 
    test_loss += loss.item()*data.size(0)
    # convert output probabilities to predicted class
    _, pred = torch.max(output, 1)
    # compare predictions to true label
    correct = np.squeeze(pred.eq(target.data.view_as(pred)))
    # calculate test accuracy for each object class
    for i in range(batch_size):
        label = target.data[i]
        class_correct[label] += correct[i].item()
        class_total[label] += 1

# calculate and print avg test loss
test_loss = test_loss/len(test_loader.dataset)
print('Test Loss: {:.6f}\n'.format(test_loss))

for i in range(10):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            str(i), 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))

### 可视化示例测试结果

此单元格会按以下格式显示测试图像及其标签：`predicted (ground-truth)`.如果准确地分类了样本，文本将为绿色；如果预测不正确，文本将为红色。


```python
# obtain one batch of test images
dataiter = iter(test_loader)
images, labels = dataiter.next()

# get sample outputs
output = model(images)
# convert output probabilities to predicted class
_, preds = torch.max(output, 1)
# prep images for display
images = images.numpy()

# plot the images in the batch, along with predicted and true labels
fig = plt.figure(figsize=(25, 4))
for idx in np.arange(20):
    ax = fig.add_subplot(2, 20/2, idx+1, xticks=[], yticks=[])
    ax.imshow(np.squeeze(images[idx]), cmap='gray')
    ax.set_title("{} ({})".format(str(preds[idx].item()), str(labels[idx].item())),
                 color=("green" if preds[idx]==labels[idx] else "red"))
```